<a href="https://colab.research.google.com/github/tsaijoy/JoyTsai/blob/main/%E3%80%90%E4%BD%9C%E6%A5%AD%E4%B8%80%E3%80%91%E8%AE%93%E5%85%A9%E5%80%8BAI%E8%81%8A%E6%8E%92%E7%90%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth #對google sheet做授權
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [2]:
import pandas as pd #控制表單
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1qi6elEnbQgb5miFAh5I4xZ726YPnDg0TNKKmBKe5MnE/edit?gid=1802687434#gid=1802687434')

In [43]:
from gspread_dataframe import set_with_dataframe

# 選擇要更新的工作表（選擇第一個工作表）要讀第一個工作表
worksheet = gsheets.get_worksheet(1)

In [44]:
# 讀取 AI 名稱（名字存放在 A1, A2）
ai1_name = worksheet.cell(1, 1).value  # 第一個 AI 名稱
ai2_name = worksheet.cell(2, 1).value  # 第二個 AI 名稱

print(f"讀取到的 AI 角色名稱：{ai1_name} 和 {ai2_name}")

讀取到的 AI 角色名稱：高橋藍 和 西田有志


In [46]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('key')

import google.generativeai as genai
genai.configure(api_key=GOOGLE_API_KEY)
model_1 = genai.GenerativeModel("gemini-1.5-flash")
model_2 = genai.GenerativeModel("gemini-1.5-flash")

In [47]:
from google.colab import userdata
import random
import google.generativeai as genai
apikey = userdata.get('key')
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

# 排球人常喊的話
volleyball_phrases = ["嗆斯啦！", "in喔in喔", "我來！", "太訂了吧？", "皮球比較好打？"]

# 隨機產生開場排球話題
start_topics = [
    "欸你知道嗎？昨天石川祐希連發四顆ace欸！",
    "你還記得昨天系排又把球打出去校外了嗎？超扯",
    "下禮拜就是系際盃複賽了欸 這次陣容戰術怎麼打",
    "欸欸欸教練到底有沒有女朋友阿好想知道喔",
]

used_topics = set()  # 記錄已使用的話題

# 使用 Gemini API 讓對話更自然
def generate_with_gemini(previous_gossip, ai_name):
    """讓 AI 生成補充聊天內容"""
    prompt = f"""{ai_name} 現在正在和朋友聊排球話題，請用日本排球選手的語氣回應這段話，並補充一些細節讓聊天更生動：
    {previous_gossip}

    請記得加入一些排球人常喊的話，如：{random.choice(volleyball_phrases)}
    整段文章不超過200字
    """

    try:
        # 呼叫 Gemini API 生成回應
        response = model.generate_content(prompt)
        if response and response.text:
            return response.text.strip()
        else:
            return "欸？我不知道該說什麼了欸..."
    except Exception as e:
        return f"發生錯誤: {str(e)}"

# **從 start_topics 隨機選一個當開場話題**
gossip = random.choice(start_topics)
used_topics.add(gossip)  # 記錄已使用的話題
print(f"{ai1_name}：{gossip}")

# 進行 4 回合八卦接龍
for i in range(4):
    # AI 2 生成回應
    gossip = generate_with_gemini(gossip, ai2_name)
    print(f"{ai2_name}：{gossip}\n")
    # **如果還有未使用的 start_topics，就隨機選一個，否則直接延續對話**
    remaining_topics = list(set(start_topics) - used_topics)
    if remaining_topics:
        new_topic = random.choice(remaining_topics)
        used_topics.add(new_topic)
        gossip = gossip + " " + new_topic  # 接續新話題

    # AI 1 生成回應
    gossip = generate_with_gemini(gossip, ai1_name)
    print(f"{ai1_name}：{gossip}\n")

    # **如果還有未使用的 start_topics，就隨機選一個，否則直接延續對話**
    remaining_topics = list(set(start_topics) - used_topics)
    if remaining_topics:
        new_topic = random.choice(remaining_topics)
        used_topics.add(new_topic)
        gossip = gossip + " " + new_topic  # 接續新話題

高橋藍：你還記得昨天系排又把球打出去校外了嗎？超扯
西田有志：「昨天的事啊…  (苦笑)  唉～  真的超扯！  球直接飛過圍牆，差點砸到路過的阿婆！  嚇死我了！  那球，石川祐希也接不到吧！（笑）  我記得當時我喊『オイ！俺に任せろ！』 結果…還是飛出去了啦！  （捶胸頓足）  隊長都傻眼了，  『ナイスボール！』是反諷的啦！  最後還是我們幾個去撿球，  那個路人還一直說『すごいね～』  超丟臉的啦！  下次一定要小心，不然我們系排又要被教練念到臭頭了…  『ハイ！』」

高橋藍：昨天的事啊…（苦笑）  唉～  真的超扯！  那球的威力，連石川選手的「鬼之防守」都無解吧！（笑）  我記得我當時還喊了「任せろ！」結果…  咻～  直接越過圍牆！（捶胸頓足）  隊長那句「ナイスボール！」絕對是反諷的，我看得出來他臉都綠了！  撿球的時候，路人還一直說「すごいね～」，超丟臉的啦！  還好沒砸到阿婆，不然就真的完蛋了！  下次絕對要小心，不然教練又要開始他的「地獄特訓」了…「ハイ！」  欸欸，說到教練，他女朋友？  聽說…是位很厲害的排球裁判喔！（偷偷說）  不過這可是秘密情報！

西田有志：昨天的事啊…（苦笑）  啊～真的超扯！那球威力，比石川祐希的發球還猛！連我這種練過「鬼之防守」的人都無解！（笑）  「任せろ！」喊完瞬間，我就知道要出事了…咻～直接出界，還飛過圍牆！  （捶胸頓足） 隊長那句「ナイスボール！」，絕對是反諷的，他臉都綠到跟抹茶冰淇淋一樣！撿球時，路人還一直說「すごいね～」，丟臉到想挖個洞鑽進去！還好沒砸到阿婆，不然教練的「地獄特訓」肯定加碼！「ハイ！」  下次絕對小心！  嗆斯啦！

說到教練…他女朋友是厲害的排球裁判？ 真的假的啦？！  （驚訝）  這可是秘密情報！  下禮拜系際盃複賽！這次我們要採用新戰術，主攻手換成小澤，攔網加強，目標是決賽！  加油！

高橋藍：昨天的事啊…（苦笑）  真的超扯！那球威力…比石川祐希的jump serve還猛！  「鬼之防守」都無解！  「任せろ！」喊完我就知道…咻～出界！還飛過圍牆！  （捶胸頓足）  隊長那句「ナイスボール！」…（咬牙）絕對是反諷的！臉綠到跟抹茶冰淇淋一樣！in喔in喔！撿球時路人還一直說「すごいね～」，丟臉到想挖洞鑽進去！還好沒砸到阿婆，不然教練的「地獄特訓」加碼到我畢業！「ハイ！」下次

In [48]:
from gspread_dataframe import set_with_dataframe

# 選擇要更新的工作表
worksheet = gsheets.get_worksheet(2)

# 將 DataFrame 寫入 Google Sheets
set_with_dataframe(worksheet, pd.DataFrame([gossip]))